In [ ]:
import pyodbc
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import os
import math
from datetime import timedelta, date
from geopy.distance import great_circle

tqdm.pandas()

SHARED_PROJECT_PATH = '...'
SHARED_PROJECT_PATH_POLY = '...'

SERVER = '...'
DATABASE = '...'
USERNAME = '...'
PASSWORD = '...'
TRUSTSERVERCERT = '...'
TRUSTEDCONN = '...'

---

## Reading and preprocessing accident alerts

In [ ]:
# Read raw accident data
connection_string = f'''
    DRIVER={{ODBC Driver 18 for SQL Server}};
    SERVER={SERVER};
    DATABASE={DATABASE};
    Trusted_Connection={TRUSTEDCONN};
    UID={USERNAME};
    PWD={PASSWORD};
    TrustServerCertificate={TRUSTSERVERCERT};
'''
sql_query = f'''
    SELECT *
    FROM ...
    WHERE type = 'ACCIDENT'
'''
conn = pyodbc.connect(connection_string)
df_acc = pd.read_sql(sql_query, conn)

In [ ]:
# Process raw accident data
df_acc = df_acc.drop_duplicates()
df_acc = df_acc.groupby('uuid').agg({
    'city': 'first',
    'confidence': 'max',
    'nThumbsUp': 'first',
    'street': 'first',
    'country': 'first',
    'subtype': 'first',
    'roadType': 'first',
    'reliability': 'max',
    'magvar': 'first',
    'reportRating': 'first',
    'ts': 'first',
    'geoWKT': 'first'
}).reset_index()

df_acc['ts'] = pd.to_datetime(df_acc['ts'])

df_acc['geometry'] = gpd.GeoSeries.from_wkt(df_acc['geoWKT'])
df_acc = gpd.GeoDataFrame(df_acc, crs='EPSG:4326', geometry=df_acc.geometry).to_crs('EPSG:23700')
df_acc.drop(columns=['geoWKT'], inplace=True)

In [ ]:
# Read Budapest polygons
gdf_poly = gpd.read_file(os.path.join(SHARED_PROJECT_PATH_POLY, 'bp_polygons_osm.geojson')).to_crs('EPSG:23700')

In [ ]:
# Filter accidents for Budapest
df_acc = df_acc[df_acc.within(gdf_poly.iloc[0].geometry)]

---

## ST-DBSCAN clustering

In [ ]:
"""
INPUTS:
    df={o1,o2,...,on} Set of objects
    spatial_threshold = Maximum geographical coordinate (spatial) distance value
    temporal_threshold = Maximum non-spatial distance value
    min_neighbors = Minimun number of points within Eps1 and Eps2 distance
OUTPUT:
    C = {c1,c2,...,ck} Set of clusters
"""
def ST_DBSCAN(df, spatial_threshold, temporal_threshold, min_neighbors):
    cluster_label = 0
    NOISE = -1
    UNMARKED = 777777
    stack = []

    # initialize each point with unmarked
    df['cluster'] = UNMARKED
    
    # for each point in database
    for index, point in tqdm(df.iterrows(), total=df.shape[0]):
        if df.loc[index]['cluster'] == UNMARKED:
            neighborhood = retrieve_neighbors(index, df, spatial_threshold, temporal_threshold)
            
            if len(neighborhood) < min_neighbors:
                df.at[index, 'cluster'] = NOISE

            else: # found a core point
                cluster_label = cluster_label + 1
                df.at[index, 'cluster'] = cluster_label# assign a label to core point

                for neig_index in neighborhood: # assign core's label to its neighborhood
                    df.at[neig_index, 'cluster'] = cluster_label
                    stack.append(neig_index) # append neighborhood to stack
                
                while len(stack) > 0: # find new neighbors from core point neighborhood
                    current_point_index = stack.pop()
                    new_neighborhood = retrieve_neighbors(current_point_index, df, spatial_threshold, temporal_threshold)
                    
                    if len(new_neighborhood) >= min_neighbors: # current_point is a new core
                        for neig_index in new_neighborhood:
                            neig_cluster = df.loc[neig_index]['cluster']
                            if (neig_cluster != NOISE) & (neig_cluster == UNMARKED): 
                                # TODO: verify cluster average before add new point
                                df.at[neig_index, 'cluster'] = cluster_label
                                stack.append(neig_index)
    return df


def retrieve_neighbors(index_center, df, spatial_threshold, temporal_threshold):
    neigborhood = []

    center_point = df.loc[index_center]

    # filter by time 
    min_time = center_point['ts'] - timedelta(minutes = temporal_threshold)
    max_time = center_point['ts'] + timedelta(minutes = temporal_threshold)
    df = df[(df['ts'] >= min_time) & (df['ts'] <= max_time)]

    # filter by distance
    for index, point in df.iterrows():
        if index != index_center:
            distance = great_circle((center_point['latitude'], center_point['longitude']), (point['latitude'], point['longitude'])).meters
            if distance <= spatial_threshold:
                neigborhood.append(index)

    return neigborhood

In [ ]:
df_acc['latitude'] = df_acc.to_crs('EPSG:4236')['geometry'].x
df_acc['longitude'] = df_acc.to_crs('EPSG:4236')['geometry'].y

In [ ]:
df_acc = ST_DBSCAN(df_acc, 300, 200, 1).drop(columns=['latitude', 'longitude'])

In [ ]:
df_acc.sort_values(by=['ts'], inplace=True)

In [ ]:
df_acc.rename(columns={'nThumbsUp': 'n_alerts_clustered'}, inplace=True)
df_acc['n_alerts_clustered'] = 1
df_acc.loc[df_acc[df_acc['subtype'] == ''].index, 'subtype'] = 'UNKNOWN'

In [ ]:
df_acc_clust = gpd.GeoDataFrame(pd.concat([
     gpd.GeoDataFrame(df_acc[df_acc['cluster'] != -1].groupby('cluster').agg({'uuid': 'first', 'city': 'first', 'confidence': 'max', 'street': 'first', 'country': 'first',
                                                                              'subtype': 'min', 'roadType': 'first', 'reliability': 'max', 'magvar': 'first',
                                                                              'reportRating': 'max', 'ts': 'first', 'geometry': 'first', 'n_alerts_clustered': 'count'}),
                     crs='EPSG:23700'),
    df_acc[df_acc['cluster'] == -1]
]), crs='EPSG:23700').sort_values('ts').drop(columns=['cluster'])                       

In [ ]:
df_acc.shape

In [ ]:
df_acc_clust.shape

In [ ]:
today = date.today().strftime('%d-%m-%Y')
out_path = os.path.join(SHARED_PROJECT_PATH, 'data', 'Waze_accidents', today)
if not os.path.exists(out_path):
    os.makedirs(out_path)
df_acc_clust.to_file(os.path.join(out_path, f'BP_acc-clust-airdist_{today}.json'), driver='GeoJSON')